In [66]:
import alborex_functions
import alborexdata
import netCDF4
import glob
import os
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap 
import numpy as np
import datetime
from importlib import reload

In [67]:
reload(alborexdata)

<module 'alborexdata' from '/home/ctroupin/Publis/201703_AlborexData/python/alborexdata.py'>

In [3]:
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

# Configuration

Create a logger

In [4]:
logger = alborexdata.configure_logging("./logs/alborexFigure4.log")

Domain

In [5]:
coordinates = [-6.75, 3.001, 34.75, 40.]
coordinates2 = [-1, -0.25, 36.65, 37.25]

Create a list of netCDF files containing SST

In [153]:
figdir = "../figures/gliders/"
sstdir = "/home/ctroupin/Data/Alborex/SST/"
datadir = "/home/ctroupin/Data/Alborex/Gliders/"

In [7]:
coastfile = "../data/coastline_cartex_f3.txt"
datafilelist = sorted(glob.glob(os.path.join(datadir, '*L1*.nc')))
nfiles = len(datafilelist)
if nfiles == 0:
    logger.warning("No SST files in directory {0}".format(datadir))
else:
    logger.info("Working on {0} data files".format(nfiles))

2018-01-05 16:29:58,060 - alborex_logger - INFO - Working on 2 data files


# Data reading and plotting

Make figure directory if necessary.

In [8]:
if os.path.isdir(figdir):
    logger.debug("Figure directory {0} already exists".format(figdir))
else:
    os.makedirs(figdir)
    logger.debug("Creating figure directory {0}".format(figdir))

2018-01-05 16:30:00,446 - alborex_logger - DEBUG - Figure directory ../figures/gliders/ already exists


## Coastline

In [9]:
coastdir = '../data/'
coastfile = os.path.join(coastdir, 'coastline_cartex_f3.txt')

if os.path.exists(coastfile):
    logger.info("{0} exists".format(coastfile))
    loncoast, latcoast = alborexdata.read_lonlat_coast(coastfile)
else:
    logger.warning("{0} does't exist".format(coastfile))  

2018-01-05 16:30:02,457 - alborex_logger - INFO - ../data/coastline_cartex_f3.txt exists


## SST

In [155]:
sstfile = os.path.join(sstdir, "A2014150020500.L2_LAC_SST4.nc")
lonsst, latsst, sst, sstqual, year, day, platform = alborex_functions.load_sst_l2(sstfile)
# Make the SST when flag > 1
sst = np.ma.masked_where(sstqual > 1, sst)

Create the projection (only once)

In [10]:
m = Basemap(projection='merc', llcrnrlon=coordinates2[0], llcrnrlat=coordinates2[2],
            urcrnrlon=coordinates2[1], urcrnrlat=coordinates2[3],
            lat_ts=0.5 * (coordinates2[2] + coordinates2[3]), resolution='h')

## Loop on the files

In [170]:
figname = "glider_tracks"
fignamesst = "glider_tracks_sst"
figtitle = "Glider tracks"
logger.info("Making figure {0}".format(figname))

fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(111)
m.ax = ax

# Add the coastline
for i in range(0, len(loncoast)):
    m.plot(np.array(loncoast[i]), np.array(latcoast[i]), 
           color='k', linewidth=.5, latlon=True)

alborexdata.add_map_grid(m, coordinates, dlon=.25, dlat=.25,
                         fontname='Times New Roman', fontsize=14, 
                         linewidth=0.2, zorder=1, color=".6")
    
colorlist = ["k", ".9"]

for ii, datafiles in enumerate(datafilelist):
    
    # logger.debug("Working on file: {0}".format(os.path.basename(datafiles)))
    
    # Read data from file
    glider = alborexdata.Glider()
    glider.get_from_netcdf(datafiles)
        
    # Remove masked values
    glider.remove_masked_coords()
    
    # Get indices for the different days
    day_indices, date_list = glider.get_day_indices(ndays=1)
    
    # Add to plot
    glider.point_plot(m, marker='o', color=colorlist[ii],
                      linestyle='-', ms=.1, zorder=3)  
    
    # Add first point
    if ii==0:
        deploymenttext = "Deployment coastal glider"
    else:
        deploymenttext = "Deployment deep glider"
        
    glider.add_initial_position(m, marker='>', markersize=10, 
                                color=colorlist[ii], label=deploymenttext)
    
    xp, yp = m(glider.lon[day_indices[:-1]], glider.lat[day_indices[:-1]])
    
    cc = 0
    for xx, yy, tt in zip(xp, yp, date_labels[:-1]):
        if np.mod(cc, 2) == 0:
            haparam = "left"
            vaparam = "bottom"
        else:
            haparam = "right"
            vaparam = "top"
            
        plt.text(xx + 1000 * (-1)** cc, yy + 1000 * (-1)** cc, tt, 
                 bbox=dict(facecolor='.95', alpha=0.85),
                 va=vaparam, ha=haparam, fontsize=16)
        cc += 1
    
    m.plot(glider.lon[day_indices[:-1]], glider.lat[day_indices[:-1]], marker='*',
           linewidth=0, color=colorlist[ii], ms=10, latlon=True)

    plt.legend(loc=2, fontsize=18, framealpha=1)
    
plt.title(figtitle, fontsize=20)

plt.savefig(os.path.join(figdir, figname), dpi=300, bbox_inches='tight')

# Add SST as background
pcm = m.pcolormesh(lonsst, latsst, sst, latlon=True, 
                   vmin=18, vmax=20.5, cmap=plt.cm.gray_r,
                   alpha=.85)

plt.savefig(os.path.join(figdir, fignamesst), dpi=300, bbox_inches='tight')
plt.close()

2018-01-05 21:10:19,141 - alborex_logger - INFO - Making figure glider_tracks
